In [86]:
import numpy as np
import tensorflow as tf
import pandas as pd

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

In [87]:
from networkarch import weight_variable, bias_variable

In [88]:
# Gather the parameters for the experiment
params = dict()

# (1) Data Parameters
params['data_name'] = 'Pendulum'
params['len_time'] = 51
n = 2  # dimension of system (and input layer)
num_initial_conditions = 5000  # per training file
params['delta_t'] = 0.02


# (2) Settings related to saving results
params['folder_name'] = 'exp2_best'


# (3) Network and Training 
params['seed'] = 17
params['widths'] = [2, 80, 80, 2]
params['dist_weights'] = ['dl', 'dl', 'dl']
params['dist_biases'] = ['', '', '']
params['scale'] = 0.1

In [91]:
# Setup the seed for the experiment
tf.set_random_seed(params['seed'])
#tf.random.set_seed(params['seed'])
np.random.seed(params['seed'])

Data Processing

In [92]:
data_orig = np.loadtxt(('./data/%s_train1_x.csv' % (params['data_name'])), delimiter=',', dtype=np.float64)
data_val = np.loadtxt(('./data/%s_val_x.csv' % (params['data_name'])), delimiter=',', dtype=np.float64)

In [93]:
# Now we would like to structure the data in the following way:
# Loop: data_orig = [x1, x2, ..., x51] -> data_x = [x1, x2, ..., x50] / data_y = [x2, x3, ..., x51]

def data_generate(data_orig):
    '''Transforms data into our learning task of predicting next step from current step.'''
    
    # Initialize size
    data_size = len(data_orig)
    num_iters = int(np.floor(data_size / params['len_time']))
    
    # Initialize new datasets
    data_x = np.zeros(((params['len_time']-1)*num_iters, 2))
    data_y = np.zeros(((params['len_time']-1)*num_iters, 2))
    
    # Loop to generate new datasets
    for i in range(num_iters):
        input_index_start = i * params['len_time'] 
        output_index_start = i * (params['len_time'] - 1)
        
        for j in range(params['len_time'] - 1):
            data_x[output_index_start + j, :] = data_orig[input_index_start + j, :]
            data_y[output_index_start + j, :] = data_orig[input_index_start + j + 1, :]
    
    return data_x, data_y


def data_extract_first_entry(data_orig):
    '''Extracts only first entry in the data in each sequence for comparison to DeepKoopman'''
    
    # Initialize size
    data_size = len(data_orig)
    num_iters = int(np.floor(data_size / params['len_time']))
    
    # Initialize dataset with first entry
    data_x = np.zeros((num_iters, 2))
    data_y = np.zeros((num_iters, 2))
    
    # Only put first entry in dataset
    
    for i in range(num_iters):
        input_index_start = i * params['len_time'] 
        
        data_x[i, :] = data_orig[i * params['len_time'], :]
        data_y[i, :] = data_orig[i * params['len_time'] + 1, :]
    
    return data_x, data_y

In [94]:
# Training Data
data_x, data_y = data_generate(data_orig)
data_x = tf.convert_to_tensor(data_x, dtype=tf.float64)
data_y = tf.convert_to_tensor(data_y, dtype=tf.float64)

# Validation Data
data_val_x, data_val_y = data_generate(data_val)
data_val_x = tf.convert_to_tensor(data_val_x, dtype=tf.float64)
data_val_y = tf.convert_to_tensor(data_val_y, dtype=tf.float64)

# Comparison Data
data_val_comp_x, data_val_comp_y = data_extract_first_entry(data_val)
data_val_comp_x = tf.convert_to_tensor(data_val_comp_x, dtype=tf.float64)
data_val_comp_y = tf.convert_to_tensor(data_val_comp_y, dtype=tf.float64)
print(data_val_comp_x.shape)
print(data_val_comp_y.shape)

(1000, 2)
(1000, 2)


In [112]:
def custom_loss(y_actual,y_pred):
    #custom_loss=tf.keras.backend.mean(tf.math.reduce_sum(tf.square(y_actual-y_pred), axis=-1))
    custom_loss=tf.math.reduce_sum(tf.math.reduce_sum(tf.square(y_actual-y_pred), axis=-1))
    return custom_loss



In [113]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(2)
])

model2 = tf.keras.Sequential([
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(80, activation='relu'),
    tf.keras.layers.Dense(2)
])

In [114]:
model.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=custom_loss)
model2.compile(optimizer=tf.keras.optimizers.Adam(0.001), loss=custom_loss)

In [115]:
for i in range(5):
    # Train
    model.trainable = True
    model.fit(data_x, data_y, epochs=1, steps_per_epoch=5, shuffle=True)
    model.save_weights('./checkpoints/my_checkpoint')
    
    # Compute the error on the validation set
    model.trainable = False
    val_loss =model.evaluate(data_val_x, data_val_y, batch_size=1, steps=5)
    print(val_loss)
    
    # Compute the 
    model2.load_weights('./checkpoints/my_checkpoint')
    val_loss_2 =model2.evaluate(data_val_comp_x, data_val_comp_y, batch_size=1, steps=5)
    print(val_loss_2)
    

Train on 5 samples
5/5 [==============================] - 8s 982ms/step - batch: 2.0000 - size: 1.0000 - loss: 423641.5125
361481.71875
2094.219970703125
Train on 5 samples
5/5 [==============================] - 5s 959ms/step - batch: 2.0000 - size: 1.0000 - loss: 317383.0750
248923.375
1442.90673828125
Train on 5 samples
5/5 [==============================] - 4s 922ms/step - batch: 2.0000 - size: 1.0000 - loss: 200624.7688
128242.3828125
746.6096801757812
Train on 5 samples
5/5 [==============================] - 5s 996ms/step - batch: 2.0000 - size: 1.0000 - loss: 85721.8828
33551.78125
195.50433349609375
Train on 5 samples
5/5 [==============================] - 4s 903ms/step - batch: 2.0000 - size: 1.0000 - loss: 19623.8814
14031.76171875
83.31536865234375


Constructing the network

In [53]:
def feedforward_network_init(widths, dist_weights, dist_biases, scale=0.1):
    """Initialize a feedforward neural network: a dictionary of weights and a dictionary of biases.

    Arguments:
        widths -- array or list of widths for layers of network
        dist_weights -- array or list of strings for distributions of weight matrices
        dist_biases -- array or list of strings for distributions of bias vectors
        scale -- (for tn distribution of weight matrices): standard deviation of normal distribution before truncation
        name -- string for prefix on weight matrices (default 'D' for decoder)

    Returns:
        weights -- dictionary of weights
        biases -- dictionary of biases

    Side effects:
        None
    """
    weights = dict()
    biases = dict()
    for i in np.arange(len(widths) - 1):
        ind = i + 1
        weights['W%d' % (ind)] = weight_variable([widths[i], widths[i + 1]], var_name='W%d' % (ind),
                                                         distribution=dist_weights[ind - 1], scale=scale)
        biases['b%d' % (ind)] = bias_variable([widths[i + 1], ], var_name='b%d' % (ind),
                                                      distribution=dist_biases[ind - 1])
    return weights, biases


def feedforward_network_apply(input_data, weights, biases, act_type, num_decoder_weights):
    """Apply a feedforward neural network to input data

    Arguments:
        input_data -- input to network
        weights -- dictionary of weights
        biases -- dictionary of biases
        act_type -- string for activation type for nonlinear layers (i.e. sigmoid, relu, or elu)
        num_weights -- number of weight matrices (layers) in the network

    Returns:
        output of decoder network applied to input prev_layer

    Side effects:
        None
    """
    prev_layer = input_data
    for i in np.arange(num_weights - 1):
        prev_layer = tf.matmul(prev_layer, weights['WD%d' % (i + 1)]) + biases['bD%d' % (i + 1)]
        if act_type == 'sigmoid':
            prev_layer = tf.sigmoid(prev_layer)
        elif act_type == 'relu':
            prev_layer = tf.nn.relu(prev_layer)
        elif act_type == 'elu':
            prev_layer = tf.nn.elu(prev_layer)

    # apply last layer without any nonlinearity
    return tf.matmul(prev_layer, weights['WD%d' % num_decoder_weights]) + biases['bD%d' % num_decoder_weights]

In [ ]:
# Defining loss function
def loss_function(data_x, data_y):
    '''Defines loss function for the problem.'''
    
    data_x
    
    
    # Initialize new datasets
    data_x = np.zeros(((params['len_time']-1)*num_iters, 2))
    data_y = np.zeros(((params['len_time']-1)*num_iters, 2))
    
    # Loop to generate new datasets
    for i in range(num_iters):
        input_index_start = i * params['len_time'] 
        output_index_start = i * (params['len_time'] - 1)
        
        for j in range(params['len_time'] - 1):
            data_x[output_index_start + j, :] = data_val[input_index_start + j, :]
            data_y[output_index_start + j, :] = data_val[input_index_start + j + 1, :]
    
    return data_x, data_y

In [58]:
# Initialize the network
weights, biases = feedforward_network(params['widths'], dist_weights=params['dist_weights'], 
                                    dist_biases=params['dist_biases'], scale=params['scale'])

# Train the network
output = feedforward_network_apply(data_x, weights, biases, 'relu', len(weights))

# Evaluating the loss
loss = np.linalg.norm(output) 